# Data Preparation

This code **should not** be run, if so, it will overwrite the previous existing data, contradicting what it was originally used for (having the same train/validation/test data to produce consistent results). The purpose of this code is to show how the data split was done.

In [1]:
import splitfolders
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from tqdm import tqdm

In [2]:
#Define the paths
dataset_path = 'flowers'
train_val_test_path = 'tvt_flowers'
train_path = os.path.join(train_val_test_path,"train")
test_path = os.path.join(train_val_test_path,"test")
val_path = os.path.join(train_val_test_path,"val")
#Path to data augmented with the CIFAR10,IMAGENET, and SVHN pre built transformations from PyTorch
train_aug_path = os.path.join(train_val_test_path,"train_aug")
#Path to data augmented with the IMAGENET pre-build transformation form Pytorch
train_less_aug_path = os.path.join(train_val_test_path,"train_less_aug")
#Path to data augmented with three different transforms - rotations, affine, flips, collor jitter, erasing
train_simple_aug_path = os.path.join(train_val_test_path,"train_simple_aug")
#Path to normalized training data
train_norm_path = os.path.join(train_val_test_path,"train_normalized")
#Path to data augmented with a single transformation containing rotations, affine, flips
train_one_aug_path = os.path.join(train_val_test_path,"train_one_aug")

In [45]:
#Split into three foldes, train, val, and test
splitfolders.ratio(dataset_path, output=train_val_test_path, seed=1337, ratio=(.72, 0.08,0.2)) 

Copying files: 4317 files [00:09, 449.54 files/s]


In [3]:
#Means and stds for normalization
channel_means = torch.Tensor([0.4590, 0.4201, 0.3005])
channel_std = torch.Tensor([0.2940, 0.2635, 0.2863])

#Define the three transformations
transformations = {}

transformations['original'] = transforms.Compose([
                                                transforms.Resize((220, 220)),
                                                transforms.ToTensor(), 
                                                transforms.Normalize((channel_means[0].item(), channel_means[1].item(), channel_means[2].item()), 
                                                                    (channel_std[0].item(), channel_std[1].item(), channel_std[2].item()))])

dataset = {}
dataset['original'] = ImageFolder(train_path, transform=transformations['original'])

In [3]:
#Pytorch Built-in Transformations
transformations['CIFAR10'] = transforms.Compose([
                                                transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
                                                transforms.Resize((220, 220)),
                                                transforms.ToTensor(), 
                                                transforms.Normalize((channel_means[0].item(), channel_means[1].item(), channel_means[2].item()), 
                                                                    (channel_std[0].item(), channel_std[1].item(), channel_std[2].item()))])


transformations['IMAGENET'] = transforms.Compose([
                                                transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
                                                transforms.Resize((220, 220)),
                                                transforms.ToTensor(), 
                                                transforms.Normalize((channel_means[0].item(), channel_means[1].item(), channel_means[2].item()), 
                                                                    (channel_std[0].item(), channel_std[1].item(), channel_std[2].item()))])

transformations['SVHN'] = transforms.Compose([
                                                transforms.AutoAugment(transforms.AutoAugmentPolicy.SVHN),
                                                transforms.Resize((220, 220)),
                                                transforms.ToTensor(), 
                                                transforms.Normalize((channel_means[0].item(), channel_means[1].item(), channel_means[2].item()), 
                                                                    (channel_std[0].item(), channel_std[1].item(), channel_std[2].item()))])

#Apply the three diferent trasnformations to the dataset and store them

dataset['CIFAR10'] = ImageFolder(train_path, transform=transformations['CIFAR10'])
dataset['IMAGENET'] = ImageFolder(train_path, transform=transformations['IMAGENET'])
dataset['SVHN'] = ImageFolder(train_path, transform=transformations['SVHN'])

In [14]:
#"Simple" Transformation

transformations['Flip_Rot_Aff'] = transforms.Compose([
                                 transforms.Resize((220, 220)),
                                 transforms.RandomHorizontalFlip(p=0.5),
                                 transforms.RandomRotation(15),
                                 transforms.RandomAffine(translate=(0.08,0.1), degrees=15),
                                 transforms.ToTensor(),
                                 transforms.Normalize((channel_means[0].item(), channel_means[1].item(), channel_means[2].item()), 
                                                                    (channel_std[0].item(), channel_std[1].item(), channel_std[2].item()))])

transformations['Random_Erasing'] = transforms.Compose([
                                 transforms.Resize((220, 220)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(10),
                                 transforms.RandomAffine(translate=(0.05,0.05), degrees=0),
                                 transforms.ToTensor(),
                                 transforms.RandomErasing(inplace=True, scale=(0.01, 0.23)),
                                 transforms.Normalize((channel_means[0].item(), channel_means[1].item(), channel_means[2].item()), 
                                                                    (channel_std[0].item(), channel_std[1].item(), channel_std[2].item()))])

transformations['Color_Jitter'] = transforms.Compose([
                                 transforms.Resize((220, 220)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(5),
                                 transforms.RandomAffine(translate=(0.05,0.05), degrees=0),
                                 transforms.ToTensor(),
                                 transforms.ColorJitter(brightness=0.2,contrast=0.2, saturation=0.2),
                                 transforms.Normalize((channel_means[0].item(), channel_means[1].item(), channel_means[2].item()), 
                                                                    (channel_std[0].item(), channel_std[1].item(), channel_std[2].item()))])
                                                
dataset['Flip_Rot_Aff'] = ImageFolder(train_path, transform=transformations['Flip_Rot_Aff'])
dataset['Random_Erasing'] = ImageFolder(train_path, transform=transformations['Random_Erasing'])
dataset['Color_Jitter'] = ImageFolder(train_path, transform=transformations['Color_Jitter'])

In [5]:
#One Augmentation Transformation

transformations['Flip_Rot_Aff_Modified'] = transforms.Compose([
                                 transforms.Resize((220, 220)),
                                 transforms.RandomHorizontalFlip(p=0.3),
                                 transforms.RandomRotation(10),
                                 transforms.RandomAffine(translate=(0.08,0.1), degrees=15),
                                 transforms.ToTensor(),
                                 transforms.Normalize((channel_means[0].item(), channel_means[1].item(), channel_means[2].item()), 
                                                                    (channel_std[0].item(), channel_std[1].item(), channel_std[2].item()))])

dataset['One_Aug'] = ImageFolder(train_path, transform=transformations['Flip_Rot_Aff_Modified'])

Generate directories to store the augmented images

In [6]:
aug_paths = [train_aug_path, train_less_aug_path, train_simple_aug_path, train_norm_path, train_one_aug_path]

for aug_path in aug_paths:
    try:
        os.makedirs(os.path.join(aug_path,"daisy"))
    except:
        print("Directory daisy already exists")

    try:
        os.makedirs(os.path.join(aug_path,"dandelion"))
    except:
        print("Directory dandelion already exists")

    try:
        os.makedirs(os.path.join(aug_path,"rose"))
    except:
        print("Directory rose already exists")

    try:
        os.makedirs(os.path.join(aug_path,"sunflower"))
    except:
        print("Directory rose already exists")

    try:
        os.makedirs(os.path.join(aug_path,"tulip"))
    except:
        print("Directory rose already exists")


Directory daisy already exists
Directory dandelion already exists
Directory rose already exists
Directory rose already exists
Directory rose already exists
Directory daisy already exists
Directory dandelion already exists
Directory rose already exists
Directory rose already exists
Directory rose already exists
Directory daisy already exists
Directory dandelion already exists
Directory rose already exists
Directory rose already exists
Directory rose already exists
Directory daisy already exists
Directory dandelion already exists
Directory rose already exists
Directory rose already exists
Directory rose already exists


The code below saves the augmented images from the 'original', 'CIFAR10', 'IMAGENET',  and 'SVHN' datasets storing in train_aug.  It also stores the images from 'original' and 'IMAGENET' datasets in another folder, train_less_aug.

In [49]:
keys = ["original", "CIFAR10", "IMAGENET", "SVHN"]

for i, key in enumerate(keys):
    for idx, data_label in tqdm(enumerate(dataset[key])):
        data, label = data_label
        if label == 0:
            path_to_save = os.path.join(train_aug_path,"daisy")
            if i == 0 or i == 2:
                save_image(data,os.path.join(train_less_aug_path,"daisy","image{}_{}.jpg".format(i,idx)))
        elif label == 1:
            path_to_save = os.path.join(train_aug_path,"dandelion")
            if i == 0 or i == 2:
                save_image(data,os.path.join(train_less_aug_path,"dandellion","image{}_{}.jpg".format(i,idx)))
        elif label == 2:
            path_to_save = os.path.join(train_aug_path,"rose")
            if i == 0 or i == 2:
                save_image(data,os.path.join(train_less_aug_path,"rose","image{}_{}.jpg".format(i,idx)))
        elif label == 3:
            path_to_save = os.path.join(train_aug_path,"sunflower")
            if i == 0 or i == 2:
                save_image(data,os.path.join(train_less_aug_path,"sunflower","image{}_{}.jpg".format(i,idx)))
        elif label == 4:
            path_to_save = os.path.join(train_aug_path,"tulip")
            if i == 0 or i == 2:
                save_image(data,os.path.join(train_less_aug_path,"tulip","image{}_{}.jpg".format(i,idx)))
    
        save_image(data,os.path.join(path_to_save,"image{}_{}.jpg".format(i,idx)))

3106it [00:29, 104.62it/s]
3106it [00:36, 85.75it/s] 
3106it [00:36, 84.77it/s] 
3106it [00:32, 97.03it/s] 


Stores the "original", "Flip_Rot_Aff", "Random_Erasing", and "Color_Jitter" datasets in train_simple_aug

In [15]:
keys = ["original", "Flip_Rot_Aff", "Random_Erasing", "Color_Jitter"]

for i, key in enumerate(keys):
    for idx, data_label in tqdm(enumerate(dataset[key])):
        data, label = data_label
        if label == 0:
            path_to_save = os.path.join(train_simple_aug_path,"daisy")
        elif label == 1:
            path_to_save = os.path.join(train_simple_aug_path,"dandelion")
        elif label == 2:
            path_to_save = os.path.join(train_simple_aug_path,"rose")
        elif label == 3:
            path_to_save = os.path.join(train_simple_aug_path,"sunflower")
        elif label == 4:
            path_to_save = os.path.join(train_simple_aug_path,"tulip")
    
        save_image(data,os.path.join(path_to_save,"image{}_{}.jpg".format(i,idx)))

3106it [00:31, 100.08it/s]
3106it [00:34, 89.78it/s] 
3106it [00:35, 87.78it/s] 
3106it [00:42, 73.35it/s]


Stores the training data normalized in the folder train_normalized

In [ ]:
for idx, data_label in tqdm(enumerate(dataset['original'])):
    data, label = data_label
    if label == 0:
        path_to_save = os.path.join(train_norm_path,"daisy")
    elif label == 1:
        path_to_save = os.path.join(train_norm_path,"dandelion")
    elif label == 2:
        path_to_save = os.path.join(train_norm_path,"rose")
    elif label == 3:
        path_to_save = os.path.join(train_norm_path,"sunflower")
    elif label == 4:
        path_to_save = os.path.join(train_norm_path,"tulip")

    save_image(data,os.path.join(path_to_save,"image{}.jpg".format(idx)))

Stores the 'original' and 'One_Aug' datasets in train_one_aug folder

In [7]:
keys = ["original", "One_Aug"]

for i, key in enumerate(keys):
    for idx, data_label in tqdm(enumerate(dataset[key])):
        data, label = data_label
        if label == 0:
                save_image(data,os.path.join(train_one_aug_path,"daisy","image{}_{}.jpg".format(i,idx)))
        elif label == 1:
                save_image(data,os.path.join(train_one_aug_path,"dandelion","image{}_{}.jpg".format(i,idx)))
        elif label == 2:
                save_image(data,os.path.join(train_one_aug_path,"rose","image{}_{}.jpg".format(i,idx)))
        elif label == 3:
                save_image(data,os.path.join(train_one_aug_path,"sunflower","image{}_{}.jpg".format(i,idx)))
        elif label == 4:
                save_image(data,os.path.join(train_one_aug_path,"tulip","image{}_{}.jpg".format(i,idx)))

3106it [00:39, 78.48it/s]
3106it [00:31, 98.60it/s] 
